In [1]:
import pandas as pd

In [2]:
train_df = pd.read_json('./train.json').reset_index()

In [3]:
val_df = pd.read_json('./val.json').reset_index()

In [4]:
test_df = pd.read_json('./test.json').reset_index()

In [5]:
val_df

,index,Unnamed: 0,review,abstractive,kategori,review_cln,ext_review
0,281,281,"Bagi Anda yang berkunjung ke Surabaya, sangat...",Tugu di depan Kebun Binatang Surabaya merupaka...,labeled-attraction,bagi anda yang berkunjung ke surabaya sangat w...,paling kalau pas ke kebon binatang baru mampir...
1,289,289,Untuk teman-teman yang sedang berada di Surab...,Taman Bungkul di Surabaya menjadi populer untu...,labeled-attraction,untuk teman teman yang sedang berada di suraba...,tapi pada saat malam hari taman ini jadi tempa...
2,104,104,Steak hut sangat menyenangkan dan saya sangat...,Restoran yang menjadi langganan keluarga karen...,labeled-restaurant,steak hut sangat menyenangkan dan saya sangat ...,lengkap enak dan yang pasti harganya terjangka...
3,278,278,Berkunjung ke kapal selam Pasopati yang kini ...,Museum Kapal Selam Pasopati di Surabaya merupa...,labeled-attraction,berkunjung ke kapal selam pasopati yang kini d...,menarik sekali bisa melihat isi dari kapal sel...
4,57,57,Sebelumnya memang sudah niat kalau ke Bandung...,GH Universal adalah hotel di Bandung yang berk...,labeled-hotel,sebelumnya memang sudah niat kalau ke bandung ...,staffnya ramah banyak senyum. [SEP] uh interio...
5,33,33,Udaya salah satu hotel favorite saya di Ubud....,The Udaya ini adalah salah satu resort terbaik...,labeled-hotel,udaya salah satu hotel favorite saya di ubud [...,kebersihan kamar dan lingkungan hotel juga san...
6,75,75,"Pengalaman pertama menginap di hotel ini, lok...",Kamarnya bersih. Secara keseluruhan fasilitas ...,labeled-hotel,pengalaman pertama menginap di hotel ini lokas...,hotelnya cukup baik dan oke makanan nya juga e...
7,76,76,kamar cukup luas dengan furniture dan dekoras...,Qunci Villas adalah hotel indah di Lombok deng...,labeled-hotel,kamar cukup luas dengan furniture dan dekorasi...,kamar cukup luas dengan furniture dan dekorasi...
8,90,90,Aeera Villa merupakan villa yang pas banget b...,Aeera Villa di Canggu menawarkan pengalaman st...,labeled-hotel,aeera villa merupakan villa yang pas banget bu...,tidak hanya dilengkapi dengan smart home syste...
9,238,238,Bila anda sudah pernah ke taman safari Puncak...,Bali Zoo adalah kebun binatang yang cukup meny...,labeled-attraction,bila anda sudah pernah ke taman safari puncak ...,ada area lain di mana pengunjung harus naik bu...


In [6]:
from datasets import Dataset

/home/python3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# !pip install transformers==4.2.1
from transformers import BertTokenizer

In [8]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)

In [9]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [10]:
alabs = pd.read_csv('raw_review.csv')

In [10]:
encoder_max_length = 512
decoder_max_length = 80

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
    inputs = tokenizer(batch["review_cln"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["abstractive"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
  
    assert all([len(x) == encoder_max_length for x in inputs.input_ids])
    assert all([len(x) == decoder_max_length for x in outputs.input_ids])
    return batch

In [11]:
batch_size=32

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    # remove_columns=["review", "extractive", "review_cln", "extractive_cln", "sent_tok", "review_key"]
)
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    # remove_columns=["review", "extractive", "review_cln", "extractive_cln", "sent_tok", "review_key"]
)

Parameter 'function'=<function process_data_to_model_inputs at 0x7f251145f9d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 1/1 [00:00<00:00,  2.72ba/s]


In [12]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [13]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("indobenchmark/indobert-base-p1","indobenchmark/indobert-base-p1")

Some weights of BertLMHeadModel were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['encoder.layer.0.crossattention.self.query.weight', 'encoder.layer.0.crossattention.self.query.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.0.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.value.weight', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.1.crossattention.self.query.weight', 'encoder.layer.1.crossattention.self.query.bias', 'encoder.layer.1.crossattention.self.key.weight', 'encoder.layer.1.crossattention.self.key.bias', 'encoder.layer.1.crossattention.self.value.weight', 'encoder.layer.1.crossattention.self.value.bias', 'encoder.layer.1.crossattention.out

In [17]:
# for layer in bert2gpt.encoder.encoder.layer[1:]:
#   for param in layer.parameters():
#     param.requires_grad = False

In [18]:
# for layer in bert2gpt.encoder.encoder.layer:
#   for param in layer.parameters():
#     print(param.requires_grad)

In [19]:
# bert2gpt

In [20]:
# bert2gpt.config.decoder.eos_token_id

In [14]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size

In [15]:
bert2bert.config.max_length = 80
bert2bert.config.min_length = 3
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 8

In [16]:
from datasets import load_metric

rouge = load_metric("rouge")

In [17]:
def rouge_matrix(pred, ref):
  result = rouge.compute(predictions=pred, references=ref)
  rouge1 = result['rouge1'].mid
  rouge2 = result['rouge2'].mid
  rougeL = result['rougeL'].mid

  return {
      "rouge1_fmeasure": round(rouge1.fmeasure, 4),
      "rouge2_fmeasure": round(rouge2.fmeasure, 4),
      "rougeL_fmeasure": round(rougeL.fmeasure, 4),
  }

In [18]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    result = rouge_matrix(pred_str, label_str)

    return result

In [19]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    do_train=True,
    do_eval=True,
    logging_steps=50,  # set to 1000 for full training
    save_steps=100,  # set to 500 for full training
    eval_steps=10, 
    warmup_steps=15,  
    num_train_epochs = 128, #TRAIN_EPOCHS
    overwrite_output_dir=True,
    save_total_limit=3
)

In [20]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Step,Training Loss
50,6.469300
100,4.343900
150,3.102700
200,2.272500
250,1.652700
300,1.184400
350,0.820600
400,0.557900
450,0.371400
500,0.250100


TrainOutput(global_step=1024, training_loss=1.0666669759666547, metrics={'train_runtime': 553.2785, 'train_samples_per_second': 1.851, 'total_flos': 30258383085895680, 'epoch': 128.0})

In [24]:
trainer.save_model('./bert2bert')

In [52]:
import os
import zipfile

zf = zipfile.ZipFile("bert2bert_128.zip", "w")
for dirname, subdirs, files in os.walk("checkpoint-1000"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()

In [41]:
from transformers import EncoderDecoderModel
model = EncoderDecoderModel.from_pretrained('checkpoint-500')

In [47]:
test_data = Dataset.from_pandas(test_df)

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["ext_review"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, max_length=80, attention_mask=attention_mask, num_beams=10,
                                repetition_penalty=5.0, num_return_sequences=1)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [48]:
resultsb = test_data.map(generate_summary, batched=True, batch_size=batch_size)
pred_strb = resultsb["pred"]
label_strb = resultsb["abstractive"]

100%|██████████| 2/2 [04:44<00:00, 142.00s/ba]


In [44]:
rouge_output = rouge_matrix(pred_strb, label_strb)
print(rouge_output)

INFO:absl:Using default tokenizer.


{'rouge1_fmeasure': 0.2822, 'rouge2_fmeasure': 0.0415, 'rougeL_fmeasure': 0.1711}


In [50]:
test_df['prediksi'] = pred_strb

In [46]:
test_df.to_csv('bert2bert-64.csv')